In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Cambiar el directorio de trabajo
%cd '/content/drive/My Drive/tfg/tfg/'

# Mostrar el directorio actual para confirmar el cambio
directorio_actual = %pwd
print("Directorio actual:", directorio_actual)

Mounted at /content/drive
/content/drive/My Drive/tfg/tfg
Directorio actual: /content/drive/My Drive/tfg/tfg


In [2]:
import os
import pandas as pd
path_actual = os.getcwd()
subdirectorio = 'datas'
file_train = 'df_train.csv'
path_train = os.path.join(path_actual, subdirectorio, file_train)

In [3]:
import pandas as pd

df_train = pd.read_csv(path_train, index_col=None)


In [4]:
df_train = df_train.drop(['trueE'],axis=1)

In [5]:
file_valid = 'df_valid.csv'
path_valid = os.path.join(path_actual, subdirectorio, file_valid)

df_valid = pd.read_csv(path_valid, index_col=None)


Este modulo será el encargado de entrenar los modelos de machine learning y ver los resultados que muetran. Además elegiremos los mejores hiperparametros para los modelos ya que estos modelos serán cargados de la biblioteca Scikit-learn.

Comprobaremos las diferencia de cada modelo para ver que modelo da mejores resultados.

Los modelos que vamos a usar son los siguientes:

*   **Regresión logistica:** Es un modelo lineal que es simple pero eficaz para problemas de clasificación binaria. Este modelo nos dará el punto de partida.

*   **Arboles de decisión:** Es un modelo versatil que puede usarse tanto para clasificación como para regresión. Los arboles de decisión son fáciles de interpretar y pueden capturar relaciones no lineales entre caracterisiticas.

*   **Random Forest:** Un ensamble de arboles de decisión. Es más robusto y preciso que un árbol unico de decisión debido a que reduce el riesgo de sobreajuste mediante el promedio de múltiples árboles

*   **SVM:** Es efectivo en espación de alta dimensión y casos donde el número de dimensiones es mayor que el número de muestras. Es útil tanto para la clasificación lineal como no lineal mediante el uso de diferentes kerneles.

*   **GBM:** Es una técnica de ensamble que construye modelos de forma secuencial, cada nuevo modelo corrige errores del modelo anterior.

*  **k-NN:** Es un modelo simple que clasifica puntos basándose en la mayoría de votos de sus k vecinos más cercanos. Es útil cuando las relaciones entre caracteristicas son complejas y no lineales.

*  **XGBoost:** es una implementación optimizada de árboles de decisión aplicados al algoritmo de Gradient Boosting. Es muy eficaz por su eficiencia en la ejecución.

*   **LightGBM:** es otro algoritmo de Gradient Boosting desarrollado por Microsoft que utiliza árboles basados en histogramas para el modelado.Utiliza la técnica de construcción de árboles basada en histogramas, que agrupa valores continuos en bins discretos



In [25]:
import numpy as np
import pandas as pd

def filtrado_datos_concatenados_optimizado1(df, N):
    kaones = []
    labels = []

    df_sorted = df.sort_values(by=['eventID', 'hitTime'], ascending=[True, False])

    for eventID, grupo in df_sorted.groupby('eventID'):
        pdgCodes = grupo['PDGcode'].unique()

        for pdgCode in pdgCodes:
            grupo_filtrado = grupo[grupo['PDGcode'] == pdgCode]
            grupo_ordenado = grupo_filtrado.head(N)

            # Separar los valores de hitX, hitY, hitZ, hitInteg
            hitX = grupo_ordenado['hitX'].values
            hitY = grupo_ordenado['hitY'].values
            hitZ = grupo_ordenado['hitZ'].values
            hitInteg = grupo_ordenado['hitInteg'].values

            # Concatenar los valores reorganizados
            hit_values_reorganized = np.concatenate([hitX, hitY, hitZ, hitInteg])

            # Aplicar padding si es necesario
            hit_values_padded = np.pad(hit_values_reorganized, (0, max(0, 4*N - len(hit_values_reorganized))), 'constant', constant_values=0)

            kaones.append(hit_values_padded)
            labels.append(pdgCode)

    return np.array(kaones), np.array(labels)


In [28]:
N=1000
matriz_train3, labels_train3 = filtrado_datos_concatenados_optimizado1(df_train, N)
print(matriz_train3[0])

[33.6069 33.6069 28.9403 ...  0.      0.      0.    ]


In [29]:
n_211 = 0
n_321 = 0

for i in labels_train3:
  if i == 211:
    n_211 +=1
  if i == 321:
    n_321+=1

print(n_321)
print(n_211)

13963
13652


In [9]:
conteo_eventID_por_PDGcode = df_train.groupby('PDGcode')['eventID'].nunique()

print(conteo_eventID_por_PDGcode)



PDGcode
211    13652
321    13963
Name: eventID, dtype: int64


In [10]:
unique_eventID_PDGCodes = df_valid.groupby('eventID').first()['PDGcode']

# Cuenta las ocurrencias de cada PDGcode en este conjunto reducido
conteo_etiquetas_unicas = unique_eventID_PDGCodes.value_counts()

print(conteo_etiquetas_unicas)


211    3901
321    3199
Name: PDGcode, dtype: int64


In [11]:
file_test = 'df_test.csv'
path_test = os.path.join(path_actual, subdirectorio, file_test)

df_test = pd.read_csv(path_test, index_col=None)

In [12]:
unique_eventID_PDGCodes = df_test.groupby('eventID').first()['PDGcode']

# Cuenta las ocurrencias de cada PDGcode en este conjunto reducido
conteo_etiquetas_unicas = unique_eventID_PDGCodes.value_counts()

print(conteo_etiquetas_unicas)

211    1951
321    1799
Name: PDGcode, dtype: int64
